# Transformers 를 이용해 Backbone 사용

## Transformers 설치
- `pip install transformers`

### Tokenizer, Model Loading
- Huggingface 모델 허브에서 제공하는 처리 모델을 다운받아 로딩한다.
- 다운로드된 모델은 `사용자 home 디렉토리\.cache\huggingface` 에 저장된다.
- 미리 학습된 언어 모델을 다운받아 사용할 때는 그 언어모델이 사용한 tokenizer를 같이 받아서 사용한다.

### [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto)
- Huggingface 에서 제공하는 다양한 모델들은 손쉽게 불러오고 사용할 수 있도록 설계된 유틸리티 클래스들을 말한다.
- 미리 학습된 특정 모델의 이름(모델 허브상에서 제공되는 이름)이나 저장된 local 경로를 제공하면 해당 모델에 맞는 적절한 클래스와 구성 요소를 자동으로 로드한다.
- 사용자는 모델을 사용하기 위한 정확한 클래스를 몰라도 쉽게 다양한 종류의 모델을 사용할 수있다.

#### 주요 Auto Class
- 기본 모델 Loading
    1. **AutoModel**
       - 주어진 모델 이름에 맞는 사전 학습된 모델을 자동으로 로드한다.
       - 예: `AutoModel.from_pretrained("bert-base-uncased")`: BERT 모델을 로드한다.
    2. **AutoTokenizer**
       - 해당 모델에 적합한 토크나이저를 자동으로 로드한다.
       - 예: `AutoTokenizer.from_pretrained("bert-base-uncased")`: BERT 모델에 맞는 토크나이저를 로드한다.
    3. **AutoConfig**
       - 모델의 설정(config)을 자동으로 로드한다. 모델 설정에는 모델의 하이퍼파라미터와 모델 구조 정보가 포함된다. 이 설정을 이용해 모델을 생성할 수있다.
       - 예: `AutoConfig.from_pretrained("bert-base-uncased")`
- Task 처리 모델 Loading
    - Pretrained backbone 모델에 각 task 에 맞는 estimator layer를 추가한 모델을 생성해 제공한다.
    - 주요 모델들
        1. **AutoModelForSequenceClassification**
           - 시퀀스(Text) 분류 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")`
        2. **AutoModelForQuestionAnswering**
           - 질문-응답 작업을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")`
        3. **AutoModelForTokenClassification**
           - 토큰 분류 작업(예: 개체명 인식)을 위한 모델을 자동으로 로드한다.
           - 예: `AutoModelForTokenClassification.from_pretrained("bert-base-uncased")`

In [1]:
from transformers import AutoModel, AutoConfig, AutoTokenizer

In [5]:
model_id = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(type(tokenizer)) # pretrained 된 토크나저저
model = AutoModel.from_pretrained(model_id)
print(type(model))# pretrained 된 모델
config = AutoConfig.from_pretrained(model_id)
print(type(config)) # model_id의 모델 설정.

<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertModel'>
<class 'transformers.models.bert.configuration_bert.BertConfig'>


In [12]:
from pprint import pprint

text = "I am a boy."
tokens = tokenizer(
    text,
    return_tensors="pt",  
    # 결과의 타입.(pt: torch.tensor, tf: tensorflow.tensor, np: ndaray, default: list)
)
pprint(tokens)
print(type(tokens['attention_mask']))
# attention_mask: 각 토큰이 실제토큰인지 (1), padding(0) 인지 구분하는 값들로 구성.
# input_ids: 토큰 id
# token_type_ids:  입력으로 두개 문장을 받았을때 몇번째 문장의 토큰인지를 구분. (0, 1)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[ 101, 1045, 2572, 1037, 2879, 1012,  102]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]])}
<class 'torch.Tensor'>


In [13]:
tokenizer.encode(text)  # 토큰 id만 반환

[101, 1045, 2572, 1037, 2879, 1012, 102]

In [23]:
## 여러 문서(문장)들을 한번에 토큰화
### -> max_length, padding, truncation 설정
### -> default 토큰 길이: 개별 문장별 길이에 맞춰 토큰생성.
text_list = ["I am a boy", "I am hungry.", "I am going to home."]
token_list = tokenizer(
    text_list, 
    max_length=7, # 최대 토큰 수
    padding=True, # padding 추가
    truncation=True # max_length가 넘을 경우 짤라낸다.
)
pprint(token_list)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1, 1, 0],
                    [1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 1045, 2572, 1037, 2879, 102, 0],
               [101, 1045, 2572, 7501, 1012, 102, 0],
               [101, 1045, 2572, 2183, 2000, 2188, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0]]}


In [24]:
token_list = tokenizer(
    text_list, 
    # max_length=7, # 최대 토큰 수
    padding=True, # max_length 설정 없이 padding=True => 제일 긴 토큰을 가진 문서에 맞춘다.
    # truncation=True # max_length가 넘을 경우 짤라낸다.
)
pprint(token_list)

{'attention_mask': [[1, 1, 1, 1, 1, 1, 0, 0],
                    [1, 1, 1, 1, 1, 1, 0, 0],
                    [1, 1, 1, 1, 1, 1, 1, 1]],
 'input_ids': [[101, 1045, 2572, 1037, 2879, 102, 0, 0],
               [101, 1045, 2572, 7501, 1012, 102, 0, 0],
               [101, 1045, 2572, 2183, 2000, 2188, 1012, 102]],
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 0]]}


In [33]:
## 토크나이저의 정보들 조회
print("총 어휘개수:", tokenizer.vocab_size, len(tokenizer))
print("token_id -> token 단어:", tokenizer.convert_ids_to_tokens(0))
print("token_id -> token 단어:", tokenizer.convert_ids_to_tokens([100, 3500, 2750]))
print('token_단어-> token_id:', tokenizer.convert_tokens_to_ids('spring'))
print('token_단어-> token_id:', tokenizer.convert_tokens_to_ids(['spring', "i", 'am']))
# tokenizer.get_vocab() # 모든 토큰들

총 어휘개수: 30522 30522
token_id -> token 단어: [PAD]
token_id -> token 단어: ['[UNK]', 'spring', 'despite']
token_단어-> token_id: 3500
token_단어-> token_id: [3500, 1045, 2572]


### tokenizer에 토큰들을 추가
- tokenizer와 모델을 같이 받았고, tokenizer에 토큰들을 추가했을 경우 이것을 모델에 적용시켜야 한다.
- 모델의 embedding vector의 크기를 재조정해야 하기 때문.

In [36]:
# 추가
print(len(tokenizer))
tokenizer.add_tokens(["미ㅏㅇ러ㅣㅏ", "ㅌ티치", "마이ㅣ치", "키이ㅏㅣ딩", "이ㅏ미칭"])
# 사이즈
print(len(tokenizer))

30532
30532


In [38]:
# 모델 업데이트
model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30532, 768, padding_idx=0)

## kcbert
- BERT 모델을 한글 텍스트로 학습 시킨 Pretrained model.
    - BERT는 Transformer의 Encoder 부분을 이용해 구현된 언어모델
    - https://arxiv.org/abs/1810.04805 
- https://huggingface.co/beomi/kcbert-base

### 토크나이저 모델 load

In [39]:
from transformers import AutoModel, AutoTokenizer

model_id = "beomi/kcbert-base"  # base model => Feature Extractor
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

In [41]:
text_list = ["안녕하세요. 반갑습니다.", 
             "kcbert는 bert 모델을 한국어로 학습한 모델입니다.", 
             "토크나이저와 모델은 같은 ID의 것으로 받아야 합니다."]

In [42]:
token_list = tokenizer(
    text_list, 
    max_length=10,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

In [43]:
print(type(token_list))

<class 'transformers.tokenization_utils_base.BatchEncoding'>


In [46]:
pprint(token_list)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'input_ids': tensor([[    2, 19017,  8482,    17,  1483,  4981,  8046,    17,     3,     0],
        [    2,    76,  4773,  4545, 13146,  4401,  4008,    67, 13146,     3],
        [    2,  3160,  4147, 16991,  4488,  4196, 16505,  4057,  8066,     3]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}


### BERT 모델을 이용해 context vector 추출
#### Model 추론결과
- **last_hidden_state**
    - 모든 token들에 대한 feature
    - 출력이 **many**인 작업에 사용한다.
- **pooler_output**
    - 입력 문장, 텍스트에 대한 context vector 이다.
    - 이 값은 **문장을 입력받아 처리하는 task**(ex: 문서분류-감정분석,문장카테고리분류, 문장유사도 분석)의 입력으로 사용한다.

In [47]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30000, 768, padding_idx=0)
    (position_embeddings): Embedding(300, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [49]:
# outputs = model(attention_mask=A, input_ids=I, token_type_ids=TI)
outputs = model(**token_list) # token_list: dict -> 가변인자에 item별로 나눠서 입력.

In [51]:
type(outputs)

transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [53]:
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [54]:
outputs['last_hidden_state'].shape
# [3: 문장수-batch, 10: 토큰수 - seq-len, 768: 개별토큰의 feature개수]

torch.Size([3, 10, 768])

In [55]:
outputs['pooler_output'].shape
# 문서(문장) 별로 하나씩 -> 문서의 context vector (문서에 대한 feature vector)의 역할.

torch.Size([3, 768])